
# SRK/T Parameter‑Fitting Notebook 🇮🇹

Questo notebook calcola **MAE** e **RMSE** della formula SRK/T “pura” sui dati
del foglio *Data* e poi ottimizza l’indice cheratometrico `n_k` e,
facoltativamente, l’indice corneale `n_e`, per ridurre l’errore.


In [ ]:
import pandas as pd

# se notebook e file xlsx sono nello stesso folder
xls_path = 'FacoDMEK.xlsx'        # <-- percorso relativo

# apri il file e mostra i nomi dei fogli per sicurezza
xls = pd.ExcelFile(xls_path)
print('Fogli disponibili:', xls.sheet_names)

# carica il foglio “Data” (rispetta maiuscole/minuscole)
df = pd.read_excel(xls, sheet_name='Data')
print(f'Dataset shape: {df.shape}')
df.head()


In [ ]:

import numpy as np

def srkt_predict(row, n_k=1.3375, n_e=1.333, n_a=1.336,
                 lcor_coeff=(3.446, 1.716, -0.0237),
                 cw_coeff=(-5.41, 0.58412, 0.098),
                 rethick_coeff=(0.65696, -0.02029),
                 offset_base=3.336, V=12.0):
    # --- inputs ---
    AL = row['Bio-AL']
    K1 = row['Bio-Ks']
    K2 = row['Bio-Kf']
    K = (K1 + K2) / 2.0  # mean K
    r = 337.5 / K        # anterior corneal radius
    
    # --- LCOR ---
    if AL <= 24.4:
        LCOR = AL
    else:
        a0, a1, a2 = lcor_coeff
        LCOR = a0 + a1 * AL + a2 * AL ** 2
    
    # --- Cw & H ---
    c0, c1, c2 = cw_coeff
    Cw = c0 + c1 * LCOR + c2 * K
    H  = r - np.sqrt(max(r**2 - (Cw**2)/4, 0))
    
    # --- ACD_est ---
    Aconst = row['A-Constant']
    ACD_const = 0.62467 * Aconst - 68.747
    offset = ACD_const - offset_base
    ACD_est = H + offset
    
    # --- RETHICK & LOPT ---
    r0, r1 = rethick_coeff
    RETHICK = r0 + r1 * AL
    LOPT = AL + RETHICK
    
    # --- thin‑lens equation ---
    ncml = n_e - 1.0
    num = 1000 * n_a * (n_a * r - ncml * LOPT)
    den = (V * (n_a * r - ncml * LOPT) + LOPT * r)
    pred_SE = num / den -               row['IOL Power'] * (LOPT - ACD_est) * (n_a * r - ncml * ACD_est) /               (n_a * (V * (n_a * r - ncml * ACD_est) + ACD_est * r))
    return pred_SE


## Baseline SRK/T

In [ ]:

from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

df['SE_pred'] = df.apply(srkt_predict, axis=1)
residuals = df['PostOP Spherical Equivalent'] - df['SE_pred']
baseline_mae  = mean_absolute_error(df['PostOP Spherical Equivalent'], df['SE_pred'])
baseline_rmse = np.sqrt(mean_squared_error(df['PostOP Spherical Equivalent'], df['SE_pred']))

print(f'Baseline MAE  : {baseline_mae:.3f} D')
print(f'Baseline RMSE : {baseline_rmse:.3f} D')


## Ottimizzazione n_k e n_e

In [ ]:

from scipy.optimize import minimize

def objective(theta):
    n_k, n_e = theta
    preds = df.apply(srkt_predict, axis=1, args=(n_k, n_e))
    return mean_squared_error(df['PostOP Spherical Equivalent'], preds)

# bounds: (1.330,1.345) for both
res = minimize(objective, x0=[1.3375, 1.333], bounds=[(1.33,1.345),(1.33,1.337)])
best_nk, best_ne = res.x
print(f'Optimal n_k: {best_nk:.5f}, Optimal n_e: {best_ne:.5f}')

df['SE_opt'] = df.apply(srkt_predict, axis=1, args=(best_nk,best_ne))
opt_mae  = mean_absolute_error(df['PostOP Spherical Equivalent'], df['SE_opt'])
opt_rmse = np.sqrt(mean_squared_error(df['PostOP Spherical Equivalent'], df['SE_opt']))
print(f'Optimized MAE : {opt_mae:.3f} D')
print(f'Optimized RMSE: {opt_rmse:.3f} D')
